# Using the DC Oscilloscope
-----

In this example notebook, we will show how to use the` Oscilloscope` class to extract time-domain data directly from the Moku at a low digitization frequency.

In [ ]:
from splendaq.daq import Oscilloscope

moku_ip = "your_ip_address"

## Data Taking for Short Durations

### The `Oscilloscope` Context Manager

As with the `LogData` class, the `Oscilloscope` class should be interacted with as a context manager, as it properly closes the connection to the Moku once complete. When entering the context manager, the user can set various global settings for the Moku, e.g. the IP address, whether to force connect to the Moku, and the acquisition mode. Below, we show the docstring of the class for the user's benefit. Note: the `force_connect` argument should be set to True as a last resort, otherwise you might be kicking another user off of the Moku!

In [ ]:
?Oscilloscope

### Setting DC Values and Taking Data

For a simple example, we show below how to take data from Input 1, where we are supplying a some DC voltage from Output 1. We'll take 0.5 seconds of data, where we are saving the data being read out by both Input 1 and Output 1.

In [ ]:
with Oscilloscope(moku_ip, force_connect=True) as Scope:
    Scope.set_input_channels(1)
    Scope.set_output_channels(1, dc_level=1)
    data_out = Scope.get_data(['Input1', 'Output1'], 0.5)

If we look at the `data_out`, we see that it's a dictionary containing the keys `'time'`, `'ch1'`, and `'ch2'`. The `'time'` field is in units of seconds, and `'ch1'` and `'ch2'` correspond to `'Input1'` and `'Output1'`, respectively. Note that this order matches what is passed to `get_data` method.

In [ ]:
data_out.keys()

Looking at the length of the data that was returned, we see that it is always 1024 bins long, such that the digitization rate is directly dependent on the duration of data that is desired.

In [ ]:
len(data_out['ch1'])

Lastly, let's plot the outputted data, just to make sure it's what we expect.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.plot(data_out['time'], data_out['ch1'])
plt.plot(data_out['time'], data_out['ch2'])
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')

We see that there is a small offset when comparing the input and the output, meaning that this should be taken into account in analyses.

## More Outputs and Inputs

The Oscilloscope can become more complex, as for example, if we were to read out the first two inputs and set DC values on the first two outputs, such as below.

In [ ]:
with Oscilloscope(moku_ip, force_connect=True) as Scope:
    Scope.set_input_channels([1, 2])
    Scope.set_output_channels([1, 2], dc_level=[1, 2])
    data_out = Scope.get_data(['Input1', 'Input2'], 0.5)

Plotting the outputs, we'll see that the two channels being plotted are Input1 and Input2, which have values around 1 V and 2 V, as expected from our settings.

In [ ]:
plt.plot(data_out['time'], data_out['ch1'])
plt.plot(data_out['time'], data_out['ch2'])
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')

From these outputted data, it is possible to do analyses such as reponses of detectors to changes in bias voltage or noise analysis. One can also use the `Sequencer` class in `splendaq` to automate, e.g., the measurement of the response of some device to various bias voltages.